**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов

In [1]:
import re
import gzip

file_name = '../data/dostoevsky-besy-p2.txt.gz'

# загружаем текст ...
with gzip.open(file_name,'rt') as f:  
    text = f.read()[105:] # ...и выкидываем заголовок

print('символов:%i\n'%(len(text)))
print(text[:364].strip())

символов:465490

Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.


In [2]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.6.7


In [3]:
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in nltk_sentence_split(text) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(text)))

sample(text,2)

предложений: 5556



[['Николай', 'Всеволодович', 'дошел', 'до', 'места', 'озабоченный', '.'],
 ['Молодежь',
  'устраивала',
  'пикники',
  ',',
  'вечеринки',
  ',',
  'иногда',
  'разъезжали',
  'по',
  'городу',
  'целою',
  'кавалькадой',
  ',',
  'в',
  'экипажах',
  'и',
  'верхами',
  '.']]

---

In [4]:
from itertools import chain
from collections import Counter

text_tokens = list(chain(*text)) # собираем все токены (слова) из текста
vocab_size = len(set(text_tokens)) # размер словаря

text_tokens_freq = Counter( text_tokens ) # оценка частоты использования слов

print('всего слов тексте: %i'%(len(text_tokens)))
print('размер словаря: %i'%(vocab_size))
text_tokens_freq.most_common()[:10] # наиболее частые токены

всего слов тексте: 92240
размер словаря: 16631


[(',', 9513),
 ('.', 4067),
 ('-', 3005),
 ('и', 2870),
 ('не', 1688),
 ('в', 1648),
 ('что', 1262),
 ('?', 893),
 ('на', 842),
 ('с', 838)]

----

In [5]:
# from nltk.util import bigrams
from nltk.util import ngrams as nltk_ngrams


# вынимаем все n-gram из текста
ngram_len = 2 # работаем с биграммами
text_ngrams = [ ngram for s in text for ngram in nltk_ngrams(s,ngram_len) ]
print('количество n-gram: %i'%(len(set(text_ngrams))))
sample(text_ngrams,5)

количество n-gram: 54227


[('в', 'галоп'),
 ('клок', ','),
 ('теперь', 'сны'),
 ('на', 'счетах'),
 (',', 'голос')]

In [6]:
# cчитаем частоту n-gram
text_ngrams_freq = Counter(text_ngrams)
sample( text_ngrams_freq.items(), 7)

[(('Он', 'медленно'), 1),
 (('и', 'обиделись'), 1),
 (('было', 'привидение'), 1),
 (('обводя', 'смелыми'), 1),
 (('мало', 'восприимчив'), 1),
 (('бледное', 'и'), 1),
 ((',', 'посвистать'), 1)]

In [7]:
text_ngrams_freq.most_common()[:10] # наиболее частые n-ngram

[((',', 'что'), 909),
 ((',', '-'), 699),
 ((',', 'и'), 495),
 ((',', 'а'), 431),
 ((',', 'но'), 351),
 ((',', 'как'), 235),
 ((',', 'я'), 209),
 ((',', 'не'), 206),
 ((',', 'в'), 189),
 (('-', 'Я'), 172)]

----

Оценка вероятностей совместного использования слов со сглаживанием Лапласа


$$
P(w_n|w_{n-1}) = \frac{ C(w_{n-1} w_{n}) +1 }{ C(w_{n-1}) + V }
$$

In [8]:
text_ngrams_prob = { # оценка совместного использования токенов
    ngram : (text_ngrams_freq[ngram]+1) / ( text_tokens_freq[ ngram[0] ] + vocab_size )
    for ngram in text_ngrams_freq 
}

text_ngrams_prob

{('Теперь', ','): 0.00012015620306398318,
 (',', 'когда'): 0.0029452264381884946,
 ('когда', 'уже'): 0.00035855145213338117,
 ('уже', 'все'): 0.00023789699060306887,
 ('все', 'прошло'): 0.00011713716762328687,
 ('прошло', ','): 0.00012023566189731875,
 (',', 'и'): 0.018971848225214197,
 ('и', 'я'): 0.0016922209117481155,
 ('я', 'пишу'): 0.000171939477303989,
 ('пишу', 'хронику'): 0.00012024289063909097,
 ('хронику', ','): 0.00018036433595863645,
 (',', 'мы'): 0.0006884944920440637,
 ('мы', 'уже'): 0.00011953143676786995,
 ('уже', 'знаем'): 0.00011894849530153443,
 ('знаем', 'в'): 0.00012022120702091849,
 ('в', 'чем'): 0.0010394441709065047,
 ('чем', 'дело'): 0.00029906094862132903,
 ('дело', ';'): 0.0002994908655286014,
 (';', 'но'): 0.004170528266913809,
 ('но', 'тогда'): 0.0001170343495816022,
 ('тогда', 'мы'): 0.00011961007116799235,
 ('мы', 'еще'): 0.00029882859191967485,
 ('еще', 'ничего'): 0.00023688262465948123,
 ('ничего', 'не'): 0.0034029850746268657,
 ('не', 'знали'): 0.00027

In [9]:
# оценка вероятности новых ngram по статистике text_ngrams_prob
def get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    if ngram in text_ngrams_prob: return text_ngrams_prob[ngram]
    token_0_freq = text_tokens_freq[ngram[0]] if ngram[0] in text_tokens_freq else 0.
    return 1./(token_0_freq+len(text_tokens_freq))

выбираем дополнение предложения с наибольшей вероятностью цепочки слов

$$
P(w_1,\ldots, w_n) = \prod_{k=1}^{n} P(w_k|w_k-1)
$$

In [10]:
from operator import mul
from functools import reduce 

# оценка предложения
def get_sentence_prob(
        sentence,
        text_ngrams_prob=text_ngrams_prob,
        text_tokens_freq=text_tokens_freq,
        ngram_len=ngram_len
    ):
    return reduce( mul, [ 
        get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in nltk_ngrams(sentence,ngram_len) 
    ] )


In [11]:
# оценка всех возможных продолжений предложения
def get_next_token_prob(sentence,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    
    sentence_prob = get_sentence_prob( # оценка предложения
            sentence,
            text_ngrams_prob=text_ngrams_prob,
            text_tokens_freq=text_tokens_freq
        )
    
    token_next = { # оценки всех возможных продолжений
        ngram[1] : 
            sentence_prob*
              get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in text_ngrams_prob if ngram[0]==sentence[-1] 
    }
    
    return token_next


In [12]:
from collections import OrderedDict

def get_top_prob_token(tokens_prob,n=3):  # n наиболее вероятных продолжений
    kf = lambda k: tokens_prob[k]
    return OrderedDict([
        (key,tokens_prob[key])
        for key in sorted(tokens_prob, key=kf, reverse=True)[:n]
    ])

----

In [13]:
# генерируем продолжения

for sentence in sample(text,30):
    if len(sentence)<10: continue
     # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    
    # генерируем возможные продолжения
    
    # считаем верояности продолжений
    next_token_prob = get_next_token_prob(sentence_) 
    
    # выбираем наиболее вероятные
    top_prob_token = get_top_prob_token(next_token_prob)
       
    print( 
        ' '.join(sentence_)
        +  ' ... ' 
        + '{ '
        + ' | '.join(top_prob_token.keys())
        + ' }'
        + '\n'
    )

Я бы никогда не мог вообразить прежде все эти ... { дни | слова | стихи }

- Да-с , ясно , но кого же он ... { , | вдруг | не }

И он , дрожа от негодования и с непомерным желанием вызова , перевел свой грозный обличительный перст на стоявшего в двух шагах ... { от | и }

Даже супруга своего поставила к Петру Степановичу в отношения почти фамилиарные , так что г. фон-Лембке жаловался ... ... { - | Я | но }

Назавтра я встретил Лизавету Николаевну верхом в сопровождении Маврикия Николаевича , выехавшую в первый ... { раз | шаг | и }

- указал он на припертую дверь в другую ... { и | комнату | , }

Капитан вытаращил глаза ; он даже и не понял ; надо было ... { , | . | бы }

Попробуйте нарисовать яблоко и положите тут же рядом настоящее яблоко - которое ... { к | однако | побудило }

Но во всяком случае , останусь ли я побежденным или победителем , я в тот же вечер возьму мою суму , нищенскую суму мою , оставлю все мои пожитки , все подарки ваши , все пенсионы и обещания будущих благ и уй